#define model

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from data_mgmt import DAO

2022-03-15 17:56:02.664018: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-15 17:56:02.664048: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def model_define(features, drop_rate):

    tf.summary.image("laser_inst", features)

    model = models.Sequential()

    model.add(layers.Conv2D(
        32, (11,1), activation="relu", padding="same", input_shape=(720,1,1),
        name="conv_1" ))
    model.add(layers.Conv2D(
        64, kernel_size=(5,1), padding="same",
        name="conv_2" ))
    model.add(layers.MaxPooling2D(
        (2,1), strides=2, padding="same",
        name="pool_1" ))
    model.add(layers.Conv2D(
        256, kernel_size=(5,1), padding="same",
        name="conv_3" ))
    model.add(layers.MaxPooling2D(
        (2,1), strides=4, padding="same",
        name="pool_2" ))

    model.add(layers.Flatten())

    model.add(layers.Dropout(drop_rate))
    model.add(layers.Dense(1024, name="dense_1"))

    model.add(layers.Dense(128, activation="relu", name="dense_2"))

    model.add(layers.Dense(3, activation="relu", name="dense_3"))

    return model

In [3]:
filename="laser_log/laser_log.csv"

data = DAO(filename)

data.read()
data.divide_data(0.25)

In [ ]:
model_cnn = model_define(data.features, 0.2)

In [5]:
model_cnn.compile(optimizer = "adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

In [6]:
model_cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_1 (Conv2D)             (None, 720, 1, 32)        384       
                                                                 
 conv_2 (Conv2D)             (None, 720, 1, 64)        10304     
                                                                 
 pool_1 (MaxPooling2D)       (None, 360, 1, 64)        0         
                                                                 
 conv_3 (Conv2D)             (None, 360, 1, 256)       82176     
                                                                 
 pool_2 (MaxPooling2D)       (None, 90, 1, 256)        0         
                                                                 
 flatten (Flatten)           (None, 23040)             0         
                                                                 
 dropout (Dropout)           (None, 23040)             0

In [6]:
history = model_cnn.fit(data.TR_features, data.TR_targets, epochs=10, validation_data=(data.TS_features, data.TS_targets))


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).